### loading all the data

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')


In [8]:
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d ahmadawad732/uadfv-dataset-new
# !unzip uadfv-dataset-new.zip

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# filepath = '/content/fake_videos/'
filepath = 'deep_fake' # for the 10gb dataset from drive

train_data = []
val_data = []
test_data = []

### DATASET 1 (FaceForensics + Celeb)
import os
for dirname, _, filenames in os.walk(filepath):
    for filename in filenames:
        if 'validation' in dirname:
            val_data.append(os.path.normpath(os.path.join(dirname, filename)))
        elif 'train' in dirname:
            train_data.append(os.path.normpath(os.path.join(dirname, filename)))
        else:
            test_data.append(os.path.normpath(os.path.join(dirname, filename)))

# Normalize paths to use forward slashes
train_data = [filepath.replace("\\", "/") for filepath in train_data]
val_data = [filepath.replace("\\", "/") for filepath in val_data]
test_data = [filepath.replace("\\", "/") for filepath in test_data]

In [10]:
os.listdir(filepath)

['test', 'train', 'validation']

In [11]:
train_data
# val_data
# test_data

['deep_fake/train/fake/000_M101.mp4',
 'deep_fake/train/fake/001_W101.mp4',
 'deep_fake/train/fake/002_M101.mp4',
 'deep_fake/train/fake/003_M101.mp4',
 'deep_fake/train/fake/004_M101.mp4',
 'deep_fake/train/fake/005_W101.mp4',
 'deep_fake/train/fake/006_M131.mp4',
 'deep_fake/train/fake/007_W101.mp4',
 'deep_fake/train/fake/008_W101.mp4',
 'deep_fake/train/fake/009_M131.mp4',
 'deep_fake/train/fake/010_W006.mp4',
 'deep_fake/train/fake/011_M131.mp4',
 'deep_fake/train/fake/012_W006.mp4',
 'deep_fake/train/fake/013_W006.mp4',
 'deep_fake/train/fake/014_M131.mp4',
 'deep_fake/train/fake/015_M109.mp4',
 'deep_fake/train/fake/016_M109.mp4',
 'deep_fake/train/fake/017_W006.mp4',
 'deep_fake/train/fake/018_W007.mp4',
 'deep_fake/train/fake/019_W007.mp4',
 'deep_fake/train/fake/020_M109.mp4',
 'deep_fake/train/fake/021_W007.mp4',
 'deep_fake/train/fake/022_M109.mp4',
 'deep_fake/train/fake/023_M113.mp4',
 'deep_fake/train/fake/024_M113.mp4',
 'deep_fake/train/fake/025_W007.mp4',
 'deep_fake/

In [12]:
# pip install tensorflow opencv-python-headless


In [13]:
train_data

['deep_fake/train/fake/000_M101.mp4',
 'deep_fake/train/fake/001_W101.mp4',
 'deep_fake/train/fake/002_M101.mp4',
 'deep_fake/train/fake/003_M101.mp4',
 'deep_fake/train/fake/004_M101.mp4',
 'deep_fake/train/fake/005_W101.mp4',
 'deep_fake/train/fake/006_M131.mp4',
 'deep_fake/train/fake/007_W101.mp4',
 'deep_fake/train/fake/008_W101.mp4',
 'deep_fake/train/fake/009_M131.mp4',
 'deep_fake/train/fake/010_W006.mp4',
 'deep_fake/train/fake/011_M131.mp4',
 'deep_fake/train/fake/012_W006.mp4',
 'deep_fake/train/fake/013_W006.mp4',
 'deep_fake/train/fake/014_M131.mp4',
 'deep_fake/train/fake/015_M109.mp4',
 'deep_fake/train/fake/016_M109.mp4',
 'deep_fake/train/fake/017_W006.mp4',
 'deep_fake/train/fake/018_W007.mp4',
 'deep_fake/train/fake/019_W007.mp4',
 'deep_fake/train/fake/020_M109.mp4',
 'deep_fake/train/fake/021_W007.mp4',
 'deep_fake/train/fake/022_M109.mp4',
 'deep_fake/train/fake/023_M113.mp4',
 'deep_fake/train/fake/024_M113.mp4',
 'deep_fake/train/fake/025_W007.mp4',
 'deep_fake/

In [14]:
import shutil
import os

# Specify the directory you want to delete
train_directory_path = 'content/train_data_frames'
val_directory_path = 'content/val_data_frames'
test_directory_path = 'content/test_data_frames'

# Check if the directory exists
def remove_directory(directory_path):
  if os.path.exists(directory_path):
    # Remove the directory and all its contents
    shutil.rmtree(directory_path)
    print(f"Directory '{directory_path}' has been deleted.")
  else:
    print(f"Directory '{directory_path}' does not exist.")

remove_directory(train_directory_path)
remove_directory(val_directory_path)
remove_directory(test_directory_path)


Directory 'content/train_data_frames' has been deleted.
Directory 'content/val_data_frames' does not exist.
Directory 'content/test_data_frames' does not exist.


### Extracting Frames and Pre-processing

In [15]:
import os
import cv2

def extract_frames_from_videos(video_dir, output_dir, frame_rate=250):
    """
    Extracts frames from videos and saves them to the output directory.

    Parameters:
    video_dir (str): Directory containing video files.
    output_dir (str): Directory to save extracted frames.
    frame_rate (int): Extract 1 frame per 'frame_rate' seconds of the video.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # for video_name in os.listdir(video_dir):
    for video_path in video_dir:
        # video_path = os.path.join(video_dir, video_name)
        cap = cv2.VideoCapture(video_path)
        count = 0

        while cap.isOpened():
            frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            ret, frame = cap.read()

            if not ret:
                break

            if frame_id % (frame_rate * int(cap.get(cv2.CAP_PROP_FPS))) == 0:
                frame_name = f"{(os.path.splitext(video_path)[0]).split('/')[-1]}_frame{frame_id}.jpg"
                frame_path = os.path.join(output_dir, frame_name)
                cv2.imwrite(frame_path, frame)

            count += 1

        cap.release()


train_directory = 'content/train_data_frames'
val_directory = 'content/val_data_frames'
test_directory = 'content/test_data_frames'
extract_frames_from_videos(train_data, train_directory)
extract_frames_from_videos(val_data, val_directory)
extract_frames_from_videos(test_data, test_directory)
# extract_frames_from_videos(filepath + '/train/real', output_directory)
# extract_frames_from_videos(filepath + '/train/fake', output_directory)


In [16]:
import tensorflow as tf

# Path to the directory containing all frames
train_frames_dir = 'content/train_data_frames'
val_frames_dir = 'content/val_data_frames'

# Define the preprocessing function
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize to a fixed size
    image = tf.image.random_flip_left_right(image)  # Data augmentation
    image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.image.per_image_standardization(image)  # Normalize the image
    return image

# Create a TensorFlow dataset
batch_size = 4
train_dataset = tf.data.Dataset.list_files(f"{train_frames_dir}/*.jpg")
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.list_files(f"{val_frames_dir}/*.jpg")
val_dataset = val_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Example of iterating through the dataset
for batch in train_dataset.take(1):
    print(batch.shape)  # Should print (batch_size, 224, 224, 3)


(4, 224, 224, 3)


### Creating the SimClR Model

In [17]:
!pip install tensorflow tensorflow-hub


Defaulting to user installation because normal site-packages is not writeable


In [18]:
train_dataset

<_PrefetchDataset element_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None)>

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Layer

# Define the encoder (EfficientNetB4) from TensorFlow Keras applications
def create_encoder():
    base_model = tf.keras.applications.EfficientNetB4(
        include_top=False,
        weights='imagenet',  # Use pretrained weights
        input_shape=(224, 224, 3),
        pooling='avg'  # Global average pooling
    )
    return base_model

# Define the projection head
def create_projection_head(encoder_output, projection_dim=128):
    x = layers.Dense(512, activation='relu')(encoder_output)
    x = layers.Dense(projection_dim)(x)
    return x

# Wrap custom layer in Layer so that it doesn't affect computation graph
def apply_normalize(projection, axis=1):
    return tf.math.l2_normalize(projection, axis=axis)

class My_normalize_layer(Layer):
    def __init__(self, axis=1, **kwargs):
        super(My_normalize_layer, self).__init__(**kwargs)
    def call(self, projection, axis=1):
        return apply_normalize(projection, axis=1)


# Create the SimCLR model
def create_simclr_model(projection_dim=128):
    inputs = layers.Input(shape=(224, 224, 3))
    encoder = create_encoder()
    features = encoder(inputs)
    projection = create_projection_head(features, projection_dim)

    # Normalize projection head output
    projection = My_normalize_layer()(projection, axis=1)

    # Build and compile the SimCLR model
    model = Model(inputs=inputs, outputs=projection, name="SimCLR_EfficientNetB4")
    return model

# Instantiate the SimCLR model, uncomment during first execution
simclr_model = create_simclr_model()
simclr_model.summary()


Model: "SimCLR_EfficientNetB4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb4 (Functional)     │ (None, 1792)           │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       918,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_normalize_layer              │ (None, 128)            │             0 │
│ (My_normalize_layer)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,657,503 (71.17 MB)

 Trainable params: 18,532,296 (70.70 MB)

 Non-trainable params: 125,207 (489.09 KB)

In [7]:
import tensorflow as tf

def nt_xent_loss(embeddings, temperature=0.5):
    """
    Compute the NT-Xent loss for a batch of embeddings.

    Parameters:
    embeddings (tf.Tensor): The embeddings from the projection head (shape: [2*batch_size, projection_dim]).
                            The first batch_size embeddings correspond to the first set of augmentations,
                            and the next batch_size correspond to the second set of augmentations.
    temperature (float): The temperature scaling factor.

    Returns:
    tf.Tensor: Scalar NT-Xent loss.
    """
    # Normalize the embeddings to have unit norm
    embeddings = tf.math.l2_normalize(embeddings, axis=1)

    # Compute the similarity matrix
    similarity_matrix = tf.matmul(embeddings, embeddings, transpose_b=True)

    # Extract positive pairs (diagonal elements from each block)
    batch_size = tf.shape(embeddings)[0] // 2
    labels = tf.range(batch_size)

    positive_mask = tf.eye(batch_size)
    positive_sim = tf.linalg.diag_part(similarity_matrix[:batch_size, batch_size:])

    # Mask out positive pairs from the similarity matrix
    similarity_matrix = similarity_matrix * (1 - tf.eye(2 * batch_size))

    # Scale by temperature
    similarity_matrix /= temperature

    # Compute the labels for cross-entropy
    labels = tf.concat([labels, labels], axis=0)

    # Compute log_softmax over the similarity matrix
    logits = similarity_matrix - tf.reduce_max(similarity_matrix, axis=1, keepdims=True)
    exp_logits = tf.exp(logits)
    log_softmax = logits - tf.math.log(tf.reduce_sum(exp_logits, axis=1, keepdims=True))

    # Calculate the NT-Xent loss
    positive_log_softmax = tf.concat([log_softmax[:batch_size, batch_size:], log_softmax[batch_size:, :batch_size]], axis=0)
    positive_log_softmax = tf.reduce_mean(tf.linalg.diag_part(positive_log_softmax))

    loss = -positive_log_softmax
    return loss


In [21]:
import os

# Directory to save the model weights
checkpoint_dir = 'content/'
os.makedirs(checkpoint_dir, exist_ok=True)


# Training parameters
num_epochs = 10
learning_rate = 0.001
temperature = 0.5

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    num_batches = 0

    # Training step
    for batch in train_dataset:
        with tf.GradientTape() as tape:
            # Forward pass
            embeddings = simclr_model(tf.concat([batch, batch], axis=0), training=True)

            # Compute the NT-Xent loss
            loss = nt_xent_loss(embeddings, temperature)

        # Backward pass
        gradients = tape.gradient(loss, simclr_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, simclr_model.trainable_variables))

        total_loss += loss
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_loss:.4f}")

    # Validation step
    total_val_loss = 0.0
    num_val_batches = 0

    for val_batch in val_dataset:
        # Forward pass for validation (no gradient tape required)
        val_embeddings = simclr_model(tf.concat([val_batch, val_batch], axis=0), training=False)

        # Compute validation NT-Xent loss
        val_loss = nt_xent_loss(val_embeddings, temperature)

        total_val_loss += val_loss
        num_val_batches += 1

    avg_val_loss = total_val_loss / num_val_batches
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")

    checkpoint_path = os.path.join(checkpoint_dir, f'simclr_epoch_{epoch + 1}.weights.h5')
    simclr_model.save_weights(checkpoint_path)
    print(f"Model weights saved to {checkpoint_path}")


Epoch 1/10, Training Loss: 0.8018
Epoch 1/10, Validation Loss: 1.8911
Model weights saved to content/simclr_epoch_1.weights.h5
Epoch 2/10, Training Loss: 0.6435
Epoch 2/10, Validation Loss: 1.9247
Model weights saved to content/simclr_epoch_2.weights.h5
Epoch 3/10, Training Loss: 0.6824
Epoch 3/10, Validation Loss: 1.8826
Model weights saved to content/simclr_epoch_3.weights.h5
Epoch 4/10, Training Loss: 0.5599
Epoch 4/10, Validation Loss: 1.6381
Model weights saved to content/simclr_epoch_4.weights.h5
Epoch 5/10, Training Loss: 0.5374
Epoch 5/10, Validation Loss: 1.3471
Model weights saved to content/simclr_epoch_5.weights.h5
Epoch 6/10, Training Loss: 0.4868
Epoch 6/10, Validation Loss: 1.0785
Model weights saved to content/simclr_epoch_6.weights.h5
Epoch 7/10, Training Loss: 0.4937
Epoch 7/10, Validation Loss: 0.9818
Model weights saved to content/simclr_epoch_7.weights.h5
Epoch 8/10, Training Loss: 0.5406
Epoch 8/10, Validation Loss: 1.6284
Model weights saved to content/simclr_epo

### Loading weights again, if you reset after traing the model

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Layer

# creating the model again from the weights i downloaded so i don't haev to train again, the model creation function is same as before,
# so adding it here again just incase
# just the run the cell above where the function is defined :)

# Load model weights
checkpoint_path = 'content/simclr_epoch_10.weights.h5'  # specify the path to your checkpoint

# now creation of model from function and addding the weights
simclr_model = create_simclr_model()
simclr_model.load_weights(checkpoint_path)
print(f'succesfully loaded weights from {checkpoint_path}')

succesfully loaded weights from content/simclr_epoch_10.weights.h5


In [23]:
# # loading the data again from local, or not, just run the above cells again

# import zipfile
# import os

# # Replace 'your_zip_file.zip' with the name of your uploaded zip file
# zip_file = 'dfd_ds.zip'

# # Define the destination directory
# destination_dir = 'dfd_ds'  # Replace with the name of your destination folder

# with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#     zip_ref.extractall(destination_dir)

# # Verify the extraction
# os.listdir(destination_dir)

In [24]:
# extract all frames with the function above where you extract the frames

## Creating the Classifier with the SimClr extractor

### Setting image_dataset_from_directory

In [25]:
import os
import shutil

def separate_videos_by_label(src_dir):
    """
    Separates videos into 'real' and 'fake' subdirectories based on filename.

    Parameters:
    - src_dir (str): Path to the source directory containing video files.
    """
    real_dir = os.path.join(src_dir, 'real')
    fake_dir = os.path.join(src_dir, 'fake')

    # Create subdirectories for 'real' and 'fake' if they don't exist
    if not os.path.exists(real_dir):
        os.makedirs(real_dir)
    if not os.path.exists(fake_dir):
        os.makedirs(fake_dir)

    # Iterate over files in the source directory
    for filename in os.listdir(src_dir):
        # Skip directories
        if os.path.isdir(os.path.join(src_dir, filename)):
            continue

        # Determine the destination directory
        if 'fake' in filename.lower():
            dest_dir = fake_dir
        else:
            dest_dir = real_dir

        # Move the file to the destination directory
        src_path = os.path.join(src_dir, filename)
        dest_path = os.path.join(dest_dir, filename)
        shutil.move(src_path, dest_path)
        print(f"Moved {filename} to {dest_dir}")

# Example usage
train_directory = 'content/train_data_frames'
val_directory = 'content/val_data_frames'
test_directory = 'content/test_data_frames'
separate_videos_by_label(train_directory)
separate_videos_by_label(val_directory)
separate_videos_by_label(test_directory)


Moved 000_M101_frame0.jpg to content/train_data_frames\real
Moved 001_W101_frame0.jpg to content/train_data_frames\real
Moved 002_M101_frame0.jpg to content/train_data_frames\real
Moved 003_M101_frame0.jpg to content/train_data_frames\real
Moved 004_M101_frame0.jpg to content/train_data_frames\real
Moved 005_W101_frame0.jpg to content/train_data_frames\real
Moved 006_M131_frame0.jpg to content/train_data_frames\real
Moved 007_W101_frame0.jpg to content/train_data_frames\real
Moved 008_W101_frame0.jpg to content/train_data_frames\real
Moved 009_M131_frame0.jpg to content/train_data_frames\real
Moved 010_W006_frame0.jpg to content/train_data_frames\real
Moved 011_M131_frame0.jpg to content/train_data_frames\real
Moved 012_W006_frame0.jpg to content/train_data_frames\real
Moved 013_W006_frame0.jpg to content/train_data_frames\real
Moved 014_M131_frame0.jpg to content/train_data_frames\real
Moved 015_M109_frame0.jpg to content/train_data_frames\real
Moved 016_M109_frame0.jpg to content/tra

In [13]:
import tensorflow as tf

batch_size = 32
img_size = (224, 224)

# Define paths to the directories
train_dir = 'content/train_data_frames'
val_dir = 'content/val_data_frames'
test_dir = 'content/test_data_frames'

# Load training dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

# Load validation dataset
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False  # Usually, no shuffling for the test set
)

# Prefetch to improve training speed
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


Found 600 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Found 250 files belonging to 2 classes.


### Creating the classifier for fine-tuning

In [27]:
print(f'layers in simclr model {len(simclr_model.layers)}')

layers in simclr model 5


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('content/fine_tuned_simCLR_with_Classifier_and_regularization.keras', monitor='val_loss', save_best_only=True)

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# freezing the weights of the sim_clr model
simclr_model.trainable = False

# Add a classification head with regularization
inputs = tf.keras.Input(shape=(224, 224, 3))
x = simclr_model(inputs, training=False)
# x = layers.BatchNormalization()(x)  # Batch normalization layer
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)  # L2 regularization
x = layers.Dropout(0.5)(x)  # Dropout for regularization
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the final model
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the classifier, uncomment this during first train
# history = model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=10,  # Adjust the number of epochs as needed
#     callbacks=[early_stopping, reduce_lr, checkpoint]  # Include callbacks for better training management
# )

# don't do this yet, because it's decreasing the accuracy, look into it later
# Unfreeze the base model for further fine-tuning
simclr_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6
                                                 ),  # Lower learning rate for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])
print('set the weights of simclr to trainable')

# Fine-tune the entire model, uncomment during first train
# history_fine = model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=5,  # Additional epochs for fine-tuning
#     initial_epoch=history.epoch[-1],
#     callbacks=[early_stopping, reduce_lr, checkpoint]  # Include callbacks for better training management
# )


set the weights of simclr to trainable


### Testing the final model

In [11]:
model.load_weights('content/fine_tuned_simCLR_with_Classifier_and_regularization.keras')

C:\Users\madha\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 1.0000 - loss: 0.3109
Test Loss: 0.3060
Test Accuracy: 1.0000


### Trying HyperParameter tuning

In [1]:
# load the fine_tued weights for prediction


# Predict on the test dataset
predictions = model.predict(test_dataset)

NameError: name 'model' is not defined

# Testing on unseen data

In [21]:
path_to_video= 'check_data'

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

filepath = 'check_data' # for the 10gb dataset from drive

check_data = []

### DATASET 1 (FaceForensics + Celeb)
import os
for dirname, _, filenames in os.walk(filepath):
    for filename in filenames:
        check_data.append(os.path.normpath(os.path.join(dirname, filename)))
        

check_data = [filepath.replace("\\", "/") for filepath in check_data]

In [26]:
check_data

['check_data/Fake/503_W134.mp4',
 'check_data/Fake/571_M115.mp4',
 'check_data/Fake/658_M124.mp4',
 'check_data/Fake/671_W005.mp4',
 'check_data/Fake/766_W027.mp4',
 'check_data/Real/M004_light_rightdown_fear_camera_front.mp4',
 'check_data/Real/M005_light_down_disgust_camera_left.mp4',
 'check_data/Real/M005_light_up_disgust_camera_left.mp4',
 'check_data/Real/M007_light_down_angry_camera_left.mp4',
 'check_data/Real/M007_light_down_surprise_camera_leftfront.mp4']

In [30]:
import os
import cv2

def extract_frames_from_videos(video_dir, output_dir, frame_rate=5):
    """
    Extracts frames from videos and saves them to the output directory.

    Parameters:
    video_dir (str): Directory containing video files.
    output_dir (str): Directory to save extracted frames.
    frame_rate (int): Extract 1 frame per 'frame_rate' seconds of the video.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # for video_name in os.listdir(video_dir):
    for video_path in video_dir:
        # video_path = os.path.join(video_dir, video_name)
        cap = cv2.VideoCapture(video_path)
        count = 0

        while cap.isOpened():
            frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            ret, frame = cap.read()

            if not ret:
                break

            if frame_id % (frame_rate * int(cap.get(cv2.CAP_PROP_FPS))) == 0:
                frame_name = f"{(os.path.splitext(video_path)[0]).split('/')[-1]}_frame{frame_id}.jpg"
                frame_path = os.path.join(output_dir, frame_name)
                cv2.imwrite(frame_path, frame)

            count += 1

        cap.release()


check_directory = 'content/check_video_frames'
extract_frames_from_videos(check_data, check_directory)

In [32]:
# put all videos inside video_frames inside a sub directory named fake
# separate_videos_by_label(video_frames_dir)

# Load the frames as a TensorFlow dataset
video_dataset = tf.keras.utils.image_dataset_from_directory(
    check_directory,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

# Predict on the video dataset
video_predictions = model.predict(video_dataset)

Found 30 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


In [35]:
# predict on model and preint accuracy
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(video_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# print the actual values vs predicted values by the model
print('actual values')
print(video_predictions)




1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2333 - loss: 1.7093
Test Loss: 1.7093
Test Accuracy: 0.2333
actual values
[[0.88638   ]
 [0.8843018 ]
 [0.87861145]
 [0.8804451 ]
 [0.8709399 ]
 [0.7631389 ]
 [0.84050584]
 [0.8048084 ]
 [0.8472265 ]
 [0.87529814]
 [0.8619117 ]
 [0.8767621 ]
 [0.8745676 ]
 [0.8580901 ]
 [0.8691559 ]
 [0.8566764 ]
 [0.84466577]
 [0.85667074]
 [0.87830603]
 [0.8637309 ]
 [0.86065876]
 [0.87919223]
 [0.87563074]
 [0.7963834 ]
 [0.87846196]
 [0.8730492 ]
 [0.8750594 ]
 [0.8847428 ]
 [0.8795038 ]
 [0.8846007 ]]
predicted values


NameError: name 'predictions' is not defined